In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

file_path = 'SanPhamMeVaBe.csv'
data = pd.read_csv(file_path)

# rename columns
data_cleaned = data.rename(columns={'Product Name': 'ProductName', 'Product Price': 'ProductPrice', 'Catergory': 'Category'})

# split data (80% train, 20% test)
X = data_cleaned[['ProductName', 'ProductPrice']]
y = data_cleaned['Category']

X_train, X_test, y_train, y_test = train_test_split(X['ProductName'], y, test_size=0.2, random_state=42)

# create pipeline
model = make_pipeline(TfidfVectorizer(), RandomForestClassifier(n_estimators=100, random_state=42))

# train model
model.fit(X_train, y_train)

# test model
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))


In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
file_path = 'SanPhamMeVaBe.csv'
data = pd.read_csv(file_path)

In [ ]:
# rename columns
data_cleaned = data.rename(columns={'Product Name': 'ProductName', 'Product Price': 'ProductPrice', 'Catergory': 'Category'})

In [ ]:
# split data (80% train, 20% test)
X = data_cleaned['ProductName'].astype(str)
y = data_cleaned['Category']

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [ ]:
# tokenization and padding
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_seq, maxlen=50, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=50, padding='post')

In [ ]:
# create model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(50000, 64),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])
model.build(input_shape=(None, 50))
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# complie model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# train model
model.fit(X_train_padded, y_train, epochs=50, validation_data=(X_test_padded, y_test))

In [ ]:

# evaluate
loss, accuracy = model.evaluate(X_test_padded, y_test)

print(f"Test Accuracy: {accuracy:.4f}")

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [5]:
data = pd.read_csv('DanCu.csv')

# change data type
data['Diện tích(km2)'] = data['Diện tích(km2)'].str.replace(',', '.').astype(float)
data['Mật độ dân số (Người/km2)'] = data['Mật độ dân số (Người/km2)'].str.replace(',', '.').astype(float)

# choose features and target
features = data[['Năm', 'Diện tích(km2)', 'Tổng_số_cặp_kết_hôn', 'Mật độ dân số (Người/km2)']]
target = data['Tỷ_lệ_sinh']

In [6]:
# split data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [7]:
# create model
model = LinearRegression()

# train model
model.fit(X_train, y_train)

LinearRegression()

In [8]:
# test model
y_pred = model.predict(X_test)

# evaluate
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

Mean Squared Error: 0.009157418126681083
R^2 Score: -0.34174624566755996


In [9]:
from sklearn.ensemble import RandomForestRegressor

# new model
model = RandomForestRegressor(random_state=42)

# train model
model.fit(X_train, y_train)

# test and evaluate again
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

Mean Squared Error: 0.009598460000000166
R^2 Score: -0.40636776556779197


In [ ]:
from sklearn.model_selection import GridSearchCV

# new model
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(RandomForestRegressor(random_state=64, n_jobs= 5, verbose=5, n_estimators=100), param_grid, cv=3)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

# evaluate again part 2
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Optimized Mean Squared Error: {mse}')
print(f'Optimized R^2 Score: {r2}')

In [11]:
from xgboost import XGBRegressor

model = XGBRegressor(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

Mean Squared Error: 0.0023691224774431037
R^2 Score: 0.6528758274808635


In [12]:
from sklearn.model_selection import RandomizedSearchCV

# new model
param_dist = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [None, 10, 20, 30],
    # 'learning_rate': [0.01, 0.1, 0.2, 0.02]
}

random_search = RandomizedSearchCV(XGBRegressor(random_state=42), param_distributions=param_dist, n_iter=15, cv=5, verbose=10)
random_search.fit(X_train, y_train)

best_model = random_search.best_estimator_

# evaluate again part 3
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Optimized Mean Squared Error: {mse}')
print(f'Optimized R^2 Score: {r2}')

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV 1/5; 1/15] START max_depth=30, n_estimators=200.............................
[CV 1/5; 1/15] END max_depth=30, n_estimators=200;, score=0.398 total time=   0.0s
[CV 2/5; 1/15] START max_depth=30, n_estimators=200.............................
[CV 2/5; 1/15] END max_depth=30, n_estimators=200;, score=0.503 total time=   0.0s
[CV 3/5; 1/15] START max_depth=30, n_estimators=200.............................
[CV 3/5; 1/15] END max_depth=30, n_estimators=200;, score=-3.312 total time=   0.0s
[CV 4/5; 1/15] START max_depth=30, n_estimators=200.............................
[CV 4/5; 1/15] END max_depth=30, n_estimators=200;, score=-22.318 total time=   0.0s
[CV 5/5; 1/15] START max_depth=30, n_estimators=200.............................
[CV 5/5; 1/15] END max_depth=30, n_estimators=200;, score=-0.307 total time=   0.0s
[CV 1/5; 2/15] START max_depth=20, n_estimators=200.............................
[CV 1/5; 2/15] END max_depth=20, n

In [13]:
import pandas as pd

data = pd.read_csv("place_info_final.csv")

In [14]:
# Define function to calculate weight
def calculate_weight(row):
    # Assign weights for each factor
    rating_weight = 0.3
    location_weight = 0.3
    hours_weight = 0.2
    chain_weight = 0.1
    district_weight = 0.1
    
    # Calculate score based on rating   
    rating_score = (row['rating'] / 5) * rating_weight
    
    # Assuming location and district scores are optimal for simplicity
    location_score = location_weight
    district_score = district_weight
    
    # Calculate score based on opening hours (assuming 14 hours is the max)
    opening_hours = 14  # Example: 8 AM to 10 PM
    hours_score = (opening_hours / 14) * hours_weight
    
    # Assuming chain score is optimal
    chain_score = chain_weight
    
    # Total score
    total_score = rating_score + location_score + hours_score + chain_score + district_score
    return total_score

In [15]:
# Apply function to calculate weight
data['weight'] = data.apply(calculate_weight, axis=1)

# sort by weight
print(data[['name', 'weight']])

                                                  name  weight
0                                          Violet Pham   0.958
1                                OVmart - Lê Trọng Tấn   0.964
2                                             Mẹ và Bé   1.000
3                                             Baby Kid   0.880
4                     Nana's House - Siêu thị Mẹ và Bé   0.934
..                                                 ...     ...
139                     KidsPlaza Store Support Office   0.940
140  KidsPlaza - Combo đồ sơ sinh mùa hè bé trai bé...   0.994
141  KidsPlaza Kiot 4 - 6 tầng 1 HH03A Khu ĐT Thanh...   1.000
142                                          KidsPlaza   1.000
143                          Kidsplaza 410 Quang Trung   0.952

[144 rows x 2 columns]


In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np
import random

In [48]:
data = pd.read_csv("SanPhamMeVaBe.csv")

In [49]:
# example of creating a column
# create a column with random values
data['Sales'] = [random.randint(50, 15000) for _ in range(len(data))]

In [50]:
# split data (80% train, 20% test)
X = data[['Product Price', 'Age Usage', 'Country', 'Trademarks']]
y = data['Sales']
product_names = data['Product Name']

In [51]:
# create encryption pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), ['Age Usage', 'Country', 'Trademarks'])
    ], remainder='passthrough')

In [52]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', LinearRegression())])

In [53]:
# train test split
X_train, X_test, y_train, y_test, product_train, product_test = train_test_split(
    X, y, product_names, test_size=0.2, random_state=42
)

In [54]:
# train model
pipeline.fit(X_train, y_train)

C:\Users\ADMIN\miniconda3\Lib\site-packages\sklearn\compose\_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat', OneHotEncoder(),
                                                  ['Age Usage', 'Country',
                                                   'Trademarks'])])),
                ('model', LinearRegression())])

In [61]:
# test model
predictions = pipeline.predict(X_test)
predictions_rounded = np.round(predictions)

# show predictions
result_df = pd.DataFrame({
    'Sản phẩm': product_test,
    'Số lượng cần nhập': predictions_rounded.astype(int)
})

# In bảng kết quả
print(result_df)

                                               Sản phẩm  Số lượng cần nhập
4740        Thiết bị giảm ốm nghén và say xe Reliefband               7661
500               Bình sữa silicon Kamidi màu nâu 250ml               7525
2225                          Giày tập đi Hanni bé trai               7515
872                               Bô vệ sinh Bibo's 105               7525
5411  Tổ yến chưng đường phèn Bestnest Hội An Kid's ...               7511
...                                                 ...                ...
5418     Sữa Aptamil Profutura Úc số 1 900g cho bé 0-6M               7547
2273       Romper bé trai Bibo's gile HTM - CA472 trắng               7516
1888    Set body cộc tay mix quần dài Bibo's ghi in gấu               7515
5361  Sữa bột Enfagrow Premium Toddler cho bé từ 1+ ...               7543
7024  Bộ quần áo dài tay cài giữa KidsPlaza Otis ND2...               7513

[1972 rows x 2 columns]


In [62]:
result_df.to_csv("predictions.csv", index=False) 